# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import os
import sys

from dotenv import load_dotenv
from pydantic import BaseModel
import datasets
from datasets import load_dataset

from loguru import logger
import pandas as pd

datasets.logging.set_verbosity_error()

sys.path.insert(0, '..')

load_dotenv()

from src.utils.data_prep import parse_dt, handle_dtypes
from src.visualization.setup import color_scheme, fsds_colors

# Controller

In [3]:
class Args(BaseModel):
    hf_dataset_path: str = "McAuley-Lab/Amazon-Reviews-2023"
    report_sample_num_rows: int = 10000
    random_seed: int = 41

args = Args()

# Load data

In [4]:
dataset = load_dataset(args.hf_dataset_path, name="0core_timestamp_Video_Games", trust_remote_code=True)
# To load all user review data such as review_text, helpful score, use: name="raw_review_All_Beauty"

In [5]:
train_raw_df = dataset['train'].to_pandas()

In [6]:
train_df = (
    train_raw_df
    .pipe(parse_dt)
    .pipe(handle_dtypes)
)

In [7]:
train_df

,user_id,parent_asin,rating,timestamp
0,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B07SRWRH5D,5.0,2020-04-16 15:31:54.941
1,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,B07DK1H3H5,4.0,2020-12-17 06:33:24.795
2,AGXVBIUFLFGMVLATYXHJYL4A5Q7Q,B07MFMFW34,5.0,2017-03-30 12:37:11.000
3,AFTC6ZR5IKNRDG5JCPVNVMU3XV2Q,B00HUWA45W,5.0,2015-03-29 01:18:52.000
4,AFTC6ZR5IKNRDG5JCPVNVMU3XV2Q,B0BCHWZX95,5.0,2019-12-29 16:40:34.017
...,...,...,...,...
3847036,AFFYUOGDNI3JXGA22RSGHT2FJZ5A,B0001AO01Y,1.0,2004-03-31 04:20:36.000
3847037,AEBGL7K4NBN2ALC6O353GJ3BIBYQ,B074QM6F7X,1.0,2018-07-12 22:12:22.360
3847038,AHTYCU6NSHJ4BY7R2YNDIDGUXS6Q,B089F1BD4W,5.0,2020-09-23 12:24:51.023
3847039,AF4KDWDGCJSUEB7JESQZ6ZWD2LNA,B0BN942894,5.0,2020-03-22 14:39:59.168


# Insert data into Postgre

In [16]:
from sqlalchemy import create_engine

In [18]:
# PostgreSQL connection details
username = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD")
host = 'localhost'
port = '5432'
database = os.environ.get("POSTGRES_DB")

# Create a connection string and engine outside the function
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# Table name where data will be inserted
table_name = 'amz_review_rating_raw'

In [28]:
# Insert data from start_date to end_date - 1 day
start_date = '2020-08-20'
end_date = '2020-08-27'

to_insert_df = train_df.loc[lambda df: df['timestamp'].between(start_date, end_date)]

to_insert_df.to_sql(table_name, engine, if_exists='append', index=False)

588